In [ ]:
using DifferentialEquations

In [ ]:
using Plots

In [ ]:
using NLsolve

In [3]:
# Inductor and capacitor design
function filter_design1(ω, D, fs, V_ref, r)
    Ts = 1/fs
    dt = D*Ts
    
    # Avg current in inductor is current thru resistor
    Irms = V_ref/r 
    
    # 30 % ripple is optimal
    di = 0.3*Irms

    v_L = 600 - 120
    # v_L = Ldi/dt
    lf = dt/di*v_L

    # ω = 1/sqrt(L*C) 
    cf = 1/(ω^2*lf)
    return lf, cf
end

filter_design1 (generic function with 1 method)

In [4]:
function filter_design2(ω, damp, r)
    cf = 1/(2*damp*r*ω)
    lf = 1/(ω^2*cf)

    return lf, cf
end

filter_design2 (generic function with 1 method)

In [5]:
function phasor_analysis(VC, ω, L, C, R)
    IR = VC/R
    IC = im*ω*C*VC
    IL = IC + IR
    VL = im*IL*ω*L
    VINV = VC + VL
    Q_L = abs(IL)^2*ω*L
    Q_C = abs(VC)^2*ω*C
    Q_INV = Q_L - Q_C
    P_INV = abs(VC)^2/R
    return Q_L, Q_C, Q_INV
end

phasor_analysis (generic function with 1 method)

In [6]:
function phasor_analysis2(VINV, ω, L, C, R)
    ZC = -im/(ω*C)
    ZRC = ZC*R/(ZC + R)
    ZL = im*ω*L
    Z = ZL + ZRC
    
    IL = VINV/Z
    VL = VINV * ZL/Z
    VRC = VINV * ZRC/Z
    IC = VRC/ZC
    IR = VRC/R

    Q_L = abs(IL)^2*ω*L
    Q_C = abs(VRC)^2*ω*C
    Q_INV = Q_L - Q_C
    P_INV = abs(VRC)^2/R
    return VRC
end

phasor_analysis2 (generic function with 1 method)

In [ ]:
v = phasor_analysis(120, 377, lf, cf, r)
rect_to_polar(v)

In [ ]:
function rect_to_polar(c)
    a = real(c)
    b = imag(c)
    return sqrt(a^2 + b^2), atan(b/a)
end

In [ ]:
function polar_to_rect(mag, angle)
    return mag*cos(angle) + im*mag*sin(angle)
end

In [2]:
# Parameters
V_dc = 1200 

f = 60
ω = 2*pi*f
D = 0.5
fs = 10e3
p_ref = 1e3
V_ref = 480
r = V_ref^2/p_ref

lf, cf = filter_design1(ω, D, fs, V_ref, r)
# damp = 0.9
# lf, cf = filter_design2(ω, damp, r)

# Power filter gain
ω_z = 1000

# Droop gains
kp = 0.01
kq = 0.0025

V_ref = 120
f_ref = 60.0
ω_ref = 2*pi*f_ref

q_L, q_C, q_ref = phasor_analysis(V_ref, ω, lf, cf, r)

# Inner control gains
kp_vd = 0.59
ki_vd = 736.0
kp_id = 1.27
ki_id = 14.3

kp_vq = 0.59
ki_vq = 736.0
kp_iq = 1.27
ki_iq = 14.3

LoadError: UndefVarError: filter_design1 not defined

In [ ]:
q_L, q_C, q_ref, p_ref, r, IL, VINV

In [ ]:
rect_to_polar(VINV)

In [ ]:
1/sqrt(lf*cf)

In [ ]:
function power_comp(vd, vq, id, iq)
    p = vd*id + vq*iq
    q = -vd*iq + vq*id
    return p, q
end

In [ ]:
function droop(pf, qf)
    ω = ω_ref - kp * (pf - p_ref)
    V = V_ref - kq * (qf - q_ref)    
    return V, ω
end

In [ ]:
function d_axis_inner_controls(vd_ref, vq_ref, vd, vq, id, iq, ϵd, γd, ϵq, γq, ω)
    # d-component controls
    dϵd = ki_vd * (vd_ref - vd) 
    id_ref = -ω*cf*vq + kp_vd * (vd_ref - vd) + ϵd
    
    dγd = ki_id * (id_ref - id)
    vd_star = kp_id * (id_ref - id) + γd - ω*lf*iq
    
    md = vd_star/(0.5*V_dc)
    
    return dϵd, dγd, md
end

In [ ]:
function q_axis_inner_controls(vd_ref, vq_ref, vd, vq, id, iq, ϵd, γd, ϵq, γq, ω)
    # q-component controls
    dϵq = ki_vq * (vq_ref - vq)   
    iq_ref = ω*cf*vd + kp_vq * (vq_ref - vq) + ϵq   
    
    dγq = ki_iq * (iq_ref - iq)
    vq_star = kp_iq * (iq_ref - iq) + γq + ω*lf*id
    
    mq = vq_star/(0.5*V_dc)
    
    return dϵq, dγq, mq
end

In [ ]:
function inverter_dynamics(vd, vq, id, iq, md, mq, ω)
    did = 1/lf * (md*V_dc/2 - vd) + ω*iq
    diq = 1/lf * (mq*V_dc/2 - vq) - ω*id
    
    dvd = 1/cf * (id - vd/r) + ω*vq
    dvq = 1/cf * (iq - vq/r) - ω*vd
    
    return did, diq, dvd, dvq
end

In [ ]:
function all_dynamics(dx, x, p, t)
    ϵd = x[1]
    ϵq = x[2]
    γd = x[3]
    γq = x[4]
    id = x[5]
    iq = x[6]
    vd = x[7]
    vq = x[8]
    pf = x[9]
    qf = x[10]
    
    # Grid-forming control
    p, q = power_comp(vd, vq, id, iq)
    V, ω = droop(pf, qf)
    
    # what is the meaning of θ? Difference angle relative to rotating frame at ω_ref

    vd_ref, vq_ref = V, 0
    
    # Inner control loops
    dϵd, dγd, md = d_axis_inner_controls(vd_ref, vq_ref, vd, vq, id, iq, ϵd, γd, ϵq, γq, ω)
    dϵq, dγq, mq = q_axis_inner_controls(vd_ref, vq_ref, vd, vq, id, iq, ϵd, γd, ϵq, γq, ω)
    
    # Inverter dynamics
    did, diq, dvd, dvq = inverter_dynamics(vd, vq, id, iq, md, mq, ω)
        
    dx[1] = dϵd
    dx[2] = dϵq
    dx[3] = dγd
    dx[4] = dγq
    dx[5] = did
    dx[6] = diq
    dx[7] = dvd
    dx[8] = dvq
    dx[9] = ω_z*(p - pf)
    dx[10] = ω_z*(q - qf)
    
    return dx, x
end

In [ ]:
x0_guess_pf = [0.3 0.0 0.1 1.0 0.1 0.0 0.1 0.1 p_ref q_ref]
res = nlsolve((dx, x) -> all_dynamics(dx, x, 0, 0), x0_guess_pf)

In [ ]:
res.zero

In [ ]:
x0 = res.zero
tspan = (0.0,4.0)
prob = ODEProblem(all_dynamics,x0,tspan)
sol = solve(prob, Rodas4(), abstol=1e-4)

In [ ]:
plot(sol,linewidth=1,title="Solution to the linear ODE with a thick line",
     xaxis="Time (t)",yaxis="Vars(t)",label="My Thick Line!") # legend=false

In [ ]:
Vd = [x[7] for x in sol.u]
Vq = [x[8] for x in sol.u]
id = [x[5] for x in sol.u]
iq = [x[6] for x in sol.u]

In [ ]:
w = [x[2] for x in droop.(Vd,Vq,id,iq)]

In [ ]:
Vq

In [ ]:
plot(sol.t, w)

In [ ]:
sol.u